In [2]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore") 

In [3]:
rail = pd.read_excel('~/rail flows/привязки жд.xlsx')
#flows_rail = pd.read_csv('~/rail flows/rail_flows_2019_test.csv', engine='python')

In [4]:
rail

,its_code,corr,direction
0,5,Минск - Москва,Обратное
1,9,Минск - Москва,Прямое
2,16,Москва - Кисловодск,Обратное
3,22,Москва - Кисловодск,Прямое
4,2205,Москва - Самара,Обратное
...,...,...,...
350,3435,Москва - Кострома,Обратное
351,3551,Москва - Вологда,Обратное
352,3590,Москва - Пенза,Обратное
353,3807,Москва - Назрань,Обратное


In [7]:
flows = pd.read_csv('/Users/eugenia/rail flows/rail_flows_2019_test.csv')

In [5]:
is_present = pd.read_excel('~/rail flows/is_pres1.xlsx')
is_present.head(1)

,№ пп,№ поезда,Сообщение,В пути,Пункт расцепки,В общем обороте,Количество составов в обороте,Unnamed: 7,Unnamed: 8,Кол-во вагонов,Электрификация участков на маршруте,Категория поезда,Признак двухэтажности,Комментарий,is_prt,is_present,its_code,Комментарий (возможный номер поезда у ИТС)
0,1,11,Москва Пав. - Анапа/Новороссийск,16:00/15:00,Рзд. 9 км,"43,125, 151,177",,6,,72,Полностью электрифицированы,Скорые сезонного и разового обращения,NaN,"Приняты без общего оборота, размер движения ус...",1.0,1,77.0,NaN


In [72]:
list_of_codes = []
additional = []
for i, r in is_present.iterrows():
    if r['is_present'] == 1:
        list_of_codes.append(r['its_code'])
        additional.append(r['Комментарий (возможный номер поезда у ИТС)'])
additional = [num for num in additional if str(num) != 'nan']
list_of_codes = [num for num in list_of_codes if str(num) != 'nan']
#all_codes = list_of_codes + additional

In [20]:
routes = list(rail['corr'].drop_duplicates())

In [62]:
%%capture
from tqdm import tqdm_notebook as tqdm
tqdm().pandas()

In [78]:
routes_and_corrs = pd.DataFrame()
with tqdm(total=len(routes)) as pbar:
    for route in routes:
        codes = list(rail[rail['corr'] == route]['its_code'])
        corrs_by_route = []
        sub_flows = flows[flows.nits_id.isin(codes)] # все поезда, относящиеся к маршруту
        sub_flows['corr'] = 0
        # колонка с корреспонденциями
        for ind, row in sub_flows.iterrows():
            sub_flows['corr'][ind] = row['plo_source_flow_name'].title() + ' - ' + row['plo_target_flow_name'].title()

        route_corrs = sub_flows.groupby(by=['plo_source_flow_name', 'plo_target_flow_name','corr', 'nits_id'], as_index=False)['flows'].sum()

        # потоки конкретных поездов внутри каждой корреспонденции

        # датафрейм:
        
        route_corrs.insert(loc=0, column='route', value=route)
        routes_and_corrs = routes_and_corrs.append(route_corrs).reset_index(drop=True)
        pbar.update(1)

In [77]:
routes_and_corrs.to_excel('routes_corrs.xlsx')

In [67]:
routes_and_corrs['in_project'] = 0
routes_and_corrs['not_included'] = 0
for ind, row in routes_and_corrs.iterrows():
    if row['nits_id'] in list_of_codes:
        routes_and_corrs['in_project'][ind] = row['flows']
    else:
        routes_and_corrs['not_included'][ind] = row['flows']

In [69]:
routes_and_corrs.to_excel('fin_routes.xlsx')

In [20]:
flows_grouped = flows.groupby(by=['plo_source_flow_name', 'plo_target_flow_name', 'nits_id'], as_index=False)['flows'].sum()
flows_grouped['corr'] = 0
for ind, row in flows_grouped.iterrows():
            flows_grouped['corr'][ind] = row['plo_source_flow_name'].title() + ' - ' + row['plo_target_flow_name'].title()

KeyboardInterrupt: 

In [28]:
routes_and_corrs['all_flows'] = 0
for ind, row in routes_and_corrs.iterrows():
    if row['nits_id'] not in list_of_codes:
        routes_and_corrs['all_flows'][ind] = flows_grouped[(flows_grouped['plo_source_flow_name'] == row['plo_source_flow_name']) & (flows_grouped['plo_target_flow_name'] == row['plo_target_flow_name']) & (flows_grouped['nits_id'] == row['nits_id'])]['flows']

KeyboardInterrupt: 

In [51]:
routes_and_corrs

,route,plo_source_flow_name,plo_target_flow_name,corr,nits_id,flows,in_project,not_included,all_flows
0,Москва - Адлер,АДЛЕР,АРМАВИР ТУАПСИНСКИЙ,Адлер - Армавир Туапсинский,858,72,72,0,0
1,Москва - Адлер,АДЛЕР,АРМАВИР ТУАПСИНСКИЙ,Адлер - Армавир Туапсинский,2146,304,0,304,304
2,Москва - Адлер,АДЛЕР,БЕЛОРЕЧЕНСКАЯ,Адлер - Белореченская,858,17,17,0,0
3,Москва - Адлер,АДЛЕР,БЕЛОРЕЧЕНСКАЯ,Адлер - Белореченская,2146,73,0,73,73
4,Москва - Адлер,АДЛЕР,БОГОЯВЛЕНСК,Адлер - Богоявленск,1112,1,0,1,1
...,...,...,...,...,...,...,...,...,...
28490,Минск - Москва,ТОЛОЧИН,МИНСК-ПАССАЖИРСКИЙ,Толочин - Минск-Пассажирский,176,3070,0,3070,0
28491,Минск - Москва,ТОЛОЧИН,МОСКВА БЕЛОРУССКАЯ,Толочин - Москва Белорусская,179,1103,1103,0,0
28492,Минск - Москва,ТОЛОЧИН,МОСКВА БЕЛОРУССКАЯ,Толочин - Москва Белорусская,959,2,0,2,0
28493,Минск - Москва,ТОЛОЧИН,ОРША-ЦЕНТРАЛЬНАЯ,Толочин - Орша-Центральная,179,1,1,0,0


In [55]:
routes_flows = pd.read_excel('/Users/eugenia/rail_flows.xlsx')
routes = routes_flows[['Маршрут', 'Корреспонденция']]
routes.columns = ['route', 'corr']

In [131]:
flows_grouped = flows.groupby(by=['plo_source_flow_name', 'plo_target_flow_name', 'nits_id'], as_index=False)['flows'].sum()
flows_grouped = flows_grouped[-flows_grouped.nits_id.isin(list_of_codes)] # здесь все ИНЫЕ поезда, есть номера поездов
flows_grouped = flows_grouped.groupby(by=['plo_source_flow_name', 'plo_target_flow_name'], as_index=False)['flows'].sum()
with tqdm(total=len(routes_and_corrs)) as pbar:
    for ind, row in routes_and_corrs.iterrows():
        try:
            routes_and_corrs['other_flows'][ind] =  flows_grouped[(flows_grouped['plo_source_flow_name'] == row['plo_source_flow_name']) & (flows_grouped['plo_target_flow_name'] == row['plo_target_flow_name'])]['flows']
        except ValueError: pass
        pbar.update(1)

In [169]:
flows_grouped1 = flows.groupby(by=['plo_source_flow_name', 'plo_target_flow_name'], as_index=False)['flows'].sum()
routes_and_corrs['all_flows'] = 0
with tqdm(total=len(routes_and_corrs)) as pbar:
    for ind, row in routes_and_corrs.iterrows():
        routes_and_corrs['all_flows'][ind] = flows_grouped1[(flows_grouped1['plo_source_flow_name'] == row['plo_source_flow_name']) & (flows_grouped1['plo_target_flow_name'] == row['plo_target_flow_name'])]['flows'].sum()
        pbar.update(1)

In [171]:
rep = routes_and_corrs[['route',
                       'corr',
                       'in_project',
                       'not_included',
                       'all_flows']]

In [ ]:
rep.columns = ['Маршрут', 
               'Корреспонденция',  
               'Потоки на маршрутах, отнесенных к проекту', 
               'Потоки на маршрутах, которые должны быть отнесены к проекту', 
               'Все потоки']

In [173]:
rep.to_excel('flows_1511.xlsx')